# Regresión Logística y Naive Bayes

Para afrontar la tarea de clasificación de emociones, hemos considerado la implementación de dos modelos de machine learning diferentes: la Regresión Logística y el Clasificador Naive Bayes Multinomial. Cada modelo se aplicará utilizando dos enfoques distintos para la extracción de características: TF-IDF (frecuencia de término-frecuencia inversa de documento) y Bag of Words (BoW).

## Regresión Logística
La Regresión Logística es un modelo estadístico que, en el contexto de clasificación, estima la probabilidad de que una instancia pertenezca a una categoría en particular. Este modelo es eficaz cuando las clases son linealmente separables. 

## Naive Bayes Multinomial
El Clasificador Naive Bayes Multinomial es especialmente adecuado para la clasificación de texto, donde las características son las frecuencias de las palabras. Este modelo se basa en el teorema de Bayes y asume independencia entre los predictores. Naive Bayes es conocido por su simplicidad y eficacia en conjuntos de datos de alta dimensión como los textos.

## Extracción de características
##### TF-IDF
TF-IDF es una técnica de extracción de características que pondera las palabras según su importancia en un documento en relación con una colección de documentos. Este método ayuda a mejorar la precisión de los modelos al dar mayor importancia a las palabras más relevantes y menos frecuentes en todos los documentos.

##### Bag of Words
Bag of Words es un modelo de representación de texto que describe la presencia de palabras dentro de un documento. No tiene en cuenta el orden o la estructura del texto, solo la frecuencia o presencia. BoW es sencillo y eficaz, aunque puede resultar en vectores de características muy grandes y esparsos.



In [1]:
#Load the libraries
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from joblib import dump
import numpy as np

In [3]:
# cargar csv
df = pd.read_csv('data/goemotions_clean.csv')
df.head()

,text,emotion
0,Shhh dont give idea,anger
1,Thank much kind stranger I really need,gratitude
2,Ion know would better buy trim make hard dose,neutral
3,Im honestly surprised We fallen much farther,excitement
4,Jurisprudence fetishist get technicality,neutral


# Regresión logística
El conjunto de datos inicial se divide en dos grupos: entrenamiento (80%) y prueba (20%). A continuación se realiza la configuración de los modelos:
+ Vectorización: Se implementaron dos métodos de vectorización:
    + BoW: Transforma los textos en vectores numéricos basados en la frecuencia de palabras y bigramas, omitiendo términos muy comunes y stop words.
    + TF-IDF: Ajusta las frecuencias de palabras basándose en su importancia relativa a lo largo del corpus, destacando palabras que son significativas para textos específicos.
+ Regresión Logística: Se configuró un modelo de Regresión Logística para cada técnica de vectorización, utilizando una penalización L2 y otros parámetros optimizados para manejar la complejidad y el sobreajuste.

Posteriormente se ha entreando cada modelo y luego se han evaluado mediante la precisión, que mide la proporción de predicciones correctas en el conjunto de prueba.


In [3]:
# Split del dataset (80-20)
X = df['text']
y = df['emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Stop words
stop_words = stopwords.words('english')

# Count vectorizer for bag of words
cv = CountVectorizer(min_df=0.01, max_df=0.95, binary=False, ngram_range=(1,2), stop_words=stop_words)

# Pipeline for BoW
pipeline_bow = Pipeline([
    ('vectorizer', cv),
    ('classifier', LogisticRegression(penalty='l2', max_iter=1000, C=1, random_state=42))
])

# Training the model for Bag of words
pipeline_bow.fit(X_train, y_train)


Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.95, min_df=0.01, ngram_range=(1, 2),
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('classifier',
                 LogisticRegression(C=1, max_iter=1000, random_state=42))])

In [5]:
# Predicting the model for bag of words
lr_bow_predict = pipeline_bow.predict(X_test)

# Accuracy score for bag of words
lr_bow_score = accuracy_score(y_test, lr_bow_predict)

In [6]:
# Tfidf vectorizer
tv = TfidfVectorizer(min_df=0.01, max_df=0.95, use_idf=True, ngram_range=(1,2), stop_words=stop_words)

# Pipeline for TF-IDF
pipeline_tfidf = Pipeline([
    ('vectorizer', tv),
    ('classifier', LogisticRegression(penalty='l2', max_iter=1000, C=1, random_state=42))
])

In [7]:
# Training the model for TF-IDF
pipeline_tfidf.fit(X_train, y_train)

# Predicting the model for TF-IDF
lr_tfidf_predict = pipeline_tfidf.predict(X_test)

# Accuracy score for TF-IDF
lr_tfidf_score = accuracy_score(y_test, lr_tfidf_predict)

In [8]:
print("lr_bow_score :", lr_bow_score)
print("lr_tfidf_score :", lr_tfidf_score)

lr_bow_score : 0.31121642969984203
lr_tfidf_score : 0.30847504878728743


In [13]:
# Guardar el modelo Bag of Words
dump(pipeline_bow, './models/logReg_bow.joblib')

# Guardar el modelo TF-IDF
dump(pipeline_tfidf, './models/logReg_tfidf.joblib')

['./models/linReg_tfidf.joblib']

Finalmente se ha llevado a cabo una prueba de predicción utilizando un nuevo texto para ver cómo cada modelo clasifica y determina la probabilidad de las emociones potenciales. Se analizan las tres principales emociones predichas para cada modelo para entender mejor sus patrones de reconocimiento y diferenciación emocional.

In [12]:
# Texto a predecir
new_text = ["I hate you"]

# Obtener las probabilidades de las clases usando el modelo Bag of Words
probabilities_bow = pipeline_bow.predict_proba(new_text)

# Obtener las probabilidades de las clases usando el modelo TF-IDF
probabilities_tfidf = pipeline_tfidf.predict_proba(new_text)

# Obtener las clases del modelo
classes = pipeline_bow.classes_

# Obtener los índices de las tres mayores probabilidades para Bag of Words
top3_indices_bow = np.argsort(probabilities_bow[0])[-3:][::-1]

# Obtener los índices de las tres mayores probabilidades para TF-IDF
top3_indices_tfidf = np.argsort(probabilities_tfidf[0])[-3:][::-1]

# Imprimir las tres emociones más probables con sus probabilidades para Bag of Words
print("Top 3 predicted emotions with probabilities using Bag of Words model:")
for index in top3_indices_bow:
    print(f"{classes[index]}: {probabilities_bow[0][index]:.2%}")

# Imprimir las tres emociones más probables con sus probabilidades para TF-IDF
print("\nTop 3 predicted emotions with probabilities using TF-IDF model:")
for index in top3_indices_tfidf:
    print(f"{classes[index]}: {probabilities_tfidf[0][index]:.2%}")


Top 3 predicted emotions with probabilities using Bag of Words model:
anger: 24.59%
neutral: 17.89%
annoyance: 11.80%

Top 3 predicted emotions with probabilities using TF-IDF model:
anger: 37.37%
annoyance: 13.09%
neutral: 11.66%


# Naive Bayes

Siguiendo un enfoque similar al utilizado en la Regresión Logística, este segmento del proyecto emplea el modelo de Naive Bayes Multinomial para la clasificación de emociones en textos, aplicando las técnicas de vectorización Bag of Words (BoW) y TF-IDF. Los  pasos seguidos son los siguientes:

### Preparación y Configuración
- **División del Conjunto de Datos**: Se divide el dataset en conjuntos de entrenamiento y prueba para validar la eficacia del modelo en datos no vistos.
- **Configuración de Vectorizadores**: Se utilizan dos métodos de vectorización, BoW y TF-IDF.
- **Modelo de Clasificación**: Se configura y entrena un clasificador Naive Bayes Multinomial para cada técnica de vectorización.

### Entrenamiento y Evaluación
- **Entrenamiento del Modelo**: Cada modelo se entrena con su respectivo conjunto de datos vectorizados.
- **Evaluación del Modelo**: Se evalúa la precisión de cada modelo utilizando el conjunto de prueba para medir la eficiencia en la clasificación de emociones.

### Predicción y Análisis
- **Predicción de Emociones**: Se realizan predicciones sobre un nuevo texto para demostrar cómo cada modelo clasifica y determina la probabilidad de las emociones.
- **Análisis de Resultados**: Se analizan y presentan las tres emociones más probables predichas por cada modelo, proporcionando una visión de la capacidad del modelo para identificar emociones específicas en un contexto dado.


In [4]:
# Split del dataset
X = df['text']
y = df['emotion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Stop words
stop_words = stopwords.words('english')

# Count vectorizer for bag of words
cv = CountVectorizer(min_df=0.01, max_df=0.95, binary=False, ngram_range=(1,2), stop_words=stop_words)

# Tfidf vectorizer
tv = TfidfVectorizer(min_df=0.01, max_df=0.95, use_idf=True, ngram_range=(1,2), stop_words=stop_words)

# Naive Bayes classifier for BoW
mnb_bow = Pipeline([
    ('vectorizer', cv),
    ('classifier', MultinomialNB())
])

# Training the model for Bag of words
mnb_bow.fit(X_train, y_train)


Pipeline(steps=[('vectorizer',
                 CountVectorizer(max_df=0.95, min_df=0.01, ngram_range=(1, 2),
                                 stop_words=['i', 'me', 'my', 'myself', 'we',
                                             'our', 'ours', 'ourselves', 'you',
                                             "you're", "you've", "you'll",
                                             "you'd", 'your', 'yours',
                                             'yourself', 'yourselves', 'he',
                                             'him', 'his', 'himself', 'she',
                                             "she's", 'her', 'hers', 'herself',
                                             'it', "it's", 'its', 'itself', ...])),
                ('classifier', MultinomialNB())])

In [6]:
# Predicting the model for bag of words
mnb_bow_predict = mnb_bow.predict(X_test)

# Accuracy score for bag of words
mnb_bow_score = accuracy_score(y_test, mnb_bow_predict)

In [7]:
# Naive Bayes classifier for TF-IDF
mnb_tfidf = Pipeline([
    ('vectorizer', tv),
    ('classifier', MultinomialNB())
])

# Training the model for TF-IDF
mnb_tfidf.fit(X_train, y_train)

# Predicting the model for TF-IDF
mnb_tfidf_predict = mnb_tfidf.predict(X_test)

# Accuracy score for TF-IDF features
mnb_tfidf_score = accuracy_score(y_test, mnb_tfidf_predict)

In [8]:
print("mnb_bow_score :", mnb_bow_score)
print("mnb_tfidf_score :", mnb_tfidf_score)

mnb_bow_score : 0.30810333612117835
mnb_tfidf_score : 0.29351361397639625


In [9]:
# Guardar el modelo Bag of Words
dump(mnb_bow, './models/naive_bow.joblib')

# Guardar el modelo TF-IDF
dump(mnb_tfidf, './models/naive_tfidf.joblib')

['./models/naive_tfidf.joblib']

In [10]:
# Texto a predecir
new_text = ["I hate you"]

# Obtener las probabilidades de las clases usando el modelo Bag of Words
probabilities_bow = mnb_bow.predict_proba(new_text)

# Obtener las probabilidades de las clases usando el modelo TF-IDF
probabilities_tfidf = mnb_tfidf.predict_proba(new_text)

# Obtener las clases del modelo
classes = mnb_bow.classes_

# Obtener los índices de las tres mayores probabilidades para Bag of Words
top3_indices_bow = np.argsort(probabilities_bow[0])[-3:][::-1]

# Obtener los índices de las tres mayores probabilidades para TF-IDF
top3_indices_tfidf = np.argsort(probabilities_tfidf[0])[-3:][::-1]

# Imprimir las tres emociones más probables con sus probabilidades para Bag of Words
print("Top 3 predicted emotions with probabilities using Bag of Words model:")
for index in top3_indices_bow:
    print(f"{classes[index]}: {probabilities_bow[0][index]:.2%}")

# Imprimir las tres emociones más probables con sus probabilidades para TF-IDF
print("\nTop 3 predicted emotions with probabilities using TF-IDF model:")
for index in top3_indices_tfidf:
    print(f"{classes[index]}: {probabilities_tfidf[0][index]:.2%}")


Top 3 predicted emotions with probabilities using Bag of Words model:
anger: 23.98%
neutral: 16.86%
annoyance: 11.85%

Top 3 predicted emotions with probabilities using TF-IDF model:
anger: 24.01%
neutral: 16.81%
annoyance: 12.33%
